In [28]:
# IMPORTACIONES
import csv
import time
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud
import geopandas as gpd
import pandas as pd
from datetime import datetime
from datetime import timedelta
from shapely import wkb

import requests

In [29]:
# CONECCION CON AMIGOCLOUD
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## PARA TRICHO

In [30]:
# PATHS
# RUTAS PARA NOTIFICACION DE LABORES DE LIBERACION DE TRICHOGRAMMA
#path_xlsx_plan = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\pd_trichogramma_2024 - copia.xlsx'

# PILA DE MENSAJES
#path_txt_msj = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'

## PARA PULVERIZACION

In [31]:
# PATHS
# RUTAS PARA NOTIFICACION DE LABORES DE LIBERACION DE TRICHOGRAMMA
path_xlsx_plan = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\CONTACTOS.xlsx'

# PILA DE MENSAJES
path_txt_msj = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'

In [32]:
# DATOS DE PILOTOS
tecnico_telf = {
    'ALEJANDRO SANCHEZ' : 78448317,
    'AURELIO GARCIA' : 73975475,
    'CHRISTIAN CALIZAYA' : 63619660,
    'EDUARDO REYES' : 75016609}

In [33]:
# EJECUTAR QUERY
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

In [34]:
# CORVERSION DE WKB A COORS
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

# NOTIFICACIONES DE LIBERACION

In [36]:
def get_registro_notificacion():
    try:
        # get data de nuevas notificacion de liberacion
        query = 'select * from dataset_354655 where procesado=false'
        notif = ejecutar_query_sql(33457, query, 'get')
        notif = notif['data']
        # extrae el primer elemento
        return notif
    except Exception as general_err:
        print(f"Error inesperado: {general_err}")
        return ''

In [37]:
def get_planificacion():
    df_plan = pd.read_excel(path_xlsx_plan)
    df_plan = df_plan[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]
    return df_plan

In [38]:
def insertar_msj_stack(idd, fecha, hora, cod, nombre, telf, msj):
    linea = f'{idd};{fecha};{hora};{cod};{nombre};{telf};{msj}'
    with open(path_txt_msj, 'r') as file:
        contenido = file.readlines()
    contenido.insert(0, '\n')
    contenido.insert(0, linea)
    with open(path_txt_msj, 'w') as file:
        file.writelines(contenido)
    print(linea)

In [39]:
def anular_registro_notificacion(idd):
    try:
        query = f'update dataset_354655 set procesado = true where id = {idd}'
        exe = ejecutar_query_sql(33457, query, 'post')
        return True
    except Exception as general_err:
        print(f"Error inesperado: {general_err}")
    return False

In [40]:
# crea el mensaje 
def crear_msj(idd, fecha, hora, cod, nombre, telfs, labor, estado, tecnico, lon, lat):
    tec_telf = tecnico_telf[tecnico]
    for i in telfs:
        msj = ''
        if estado == 'INICIO DE LABOR':
            msj = f'''Señor(a) *{nombre}*, espero que se encuentre bien, se le informa que se esta iniciando el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Este mensaje a sido generado de manera automatica por el equipo de campo. Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        elif estado == 'FIN DE LABOR':
            msj = f'''Señor(a) *{nombre}*, espero que se encuentre bien, se le informa que se ha finalizado el trabajo de *{labor}* en la siguiente ubicacion: \
https://www.google.com/maps?q={lat},{lon}. \
Este mensaje a sido generado de manera automatica por el equipo de campo. Para cualquier consulta puede comunicarse con ING. {tecnico} al {tec_telf}'''
        insertar_msj_stack(idd, fecha, hora, cod, nombre, i, msj)

In [41]:
reg_amigo = get_registro_notificacion()

In [42]:
df_plan=get_planificacion()

In [43]:
def revisar_nuevos_mensajes():
    reg_amigo = get_registro_notificacion()
    if len(reg_amigo) == 0:
        # print('No existen notificaciones nuevas')
        return False
    reg_amigo = reg_amigo[0]
    # extrae cod_ca y nom_ca del registro
    cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
    nom_ca = reg_amigo['canhero'].split(' / ')[1]
    # filtra en la lista de planificacion el cod_ca para obtener los numeros cel.
    df_plan=get_planificacion()
    reg = df_plan[df_plan['cod_ca']==cod_ca].iloc[0]
    # id del registro en amigocloud
    idd = reg_amigo['id']
    # extrae fecha y hora de registro menos 4 horas por desfase temprotal
    fecha_hora_obj = datetime.fromisoformat(reg_amigo['fecha_registro'])
    fecha_hora_menos_4h = fecha_hora_obj - timedelta(hours=4)
    # formatea la fecha de registro
    fecha_str = fecha_hora_menos_4h.strftime('%d-%m-%Y')
    # formatea la hora de registro
    hora_str = fecha_hora_menos_4h.strftime('%H:%M:%S')
    # extrae los numeros telefonicos
    telefonos = [reg['telf01'], reg['telf02'], reg['telf03']]
    # valida los numeros telefonicos
    
    telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
    #telefonos_validos = []
    
    # extrae nombre del piloto
    tecnico = reg_amigo['piloto']
    tec_telf = tecnico_telf[tecnico]
    telefonos_validos.append(tec_telf)
    telefonos_validos.append(68908131) #jose
    telefonos_validos.append(75380725) #mario
    telefonos_validos.append(77437601) #karina
    telefonos_validos.append(78194371) #bismark
    
    # extra el tipo de labor (liberacion o pulverizacion)
    labor = reg_amigo['tipo_labor']
    # extrae el estado de la labor (inicio o fin de labor)
    estado = reg_amigo['tipo_mensaje']
    # extrae y convierte la ubicacion
    ubi = convertir_wkb(reg_amigo['ubicacion'])
    longitud = ubi.x  # Longitud (coordenada X)
    latitud = ubi.y   # Latitud (coordenada Y)
    crear_msj(idd, fecha_str, hora_str, cod_ca, nom_ca, telefonos_validos, labor, estado, tecnico, longitud, latitud)
    anular_registro_notificacion(idd)
    return True
    ##time.sleep(30)

In [44]:
def sendMessage(para, mensaje):
    url = "http://localhost:8080/message/sendText/UTEA-CHAT"
    payload = {
        "number": '591' + para,
        "text": mensaje,
    }
    headers = {
        "apikey": "D6CD36F3CE20-4962-A175-8D5583A345D9",
        "Content-Type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)

In [45]:
path_txt_msj = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_stack.txt'
path_txt_log = r'C:\Users\Usuario\Desktop\PYTHON\utea_scripts\notificaciones_whastapp\_msj_log.txt'

In [46]:
def get_msj_from_stack():
    # Abrir el archivo en modo lectura y escritura
    with open(path_txt_msj, 'r+', newline='') as archivo:
        lector = csv.reader(archivo, delimiter=';')
    
        # Convertir el contenido a una lista
        lineas = list(lector)
        
        # Verificar si hay líneas en el archivo
        if lineas:
            # Extraer la primera línea
            primera_linea = lineas[0]
        
            # Formatear la primera línea en un diccionario
            encabezados = ['id', 'fecha', 'hora', 'codigo', 'nombre', 'numero_celular', 'mensaje']
            diccionario_primera_linea = dict(zip(encabezados, primera_linea))
        
            # Eliminar la primera línea de la lista
            lineas = lineas[1:]
        
            # Reposicionar el puntero al inicio del archivo y truncar su contenido
            archivo.seek(0)
            archivo.truncate()
        
            # Reescribir las líneas restantes en el archivo
            escritor = csv.writer(archivo, delimiter=';')
            escritor.writerows(lineas)
        
            # Guardar la línea eliminada en el archivo de log en el mismo formato
            with open(path_txt_log, 'a') as log:
                log.write(';'.join(primera_linea) + '\n')
        
            return diccionario_primera_linea
        else:
            return {}

In [47]:
def enviar_mensaje():
    msj = get_msj_from_stack()
    #print(len(msj))
    if len(msj) > 0:
        sendMessage(msj['numero_celular'], msj['mensaje'])
        return True
    else:
        return False
        #print('No se tiene mensajes nuevos para enviar')

In [ ]:
while True:
    x = revisar_nuevos_mensajes()
    if x == True:
        while True:
            envio = enviar_mensaje()
            if envio == False:
                break
    time.sleep(600)

397;07-03-2025;01:01:32;41594;AGROP. CAMPODULCE SRL;73643129;Señor(a) *AGROP. CAMPODULCE SRL*, espero que se encuentre bien, se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.235721861668225,-62.97494216571519. Este mensaje a sido generado de manera automatica por el equipo de campo. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
397;07-03-2025;01:01:32;41594;AGROP. CAMPODULCE SRL;78448317;Señor(a) *AGROP. CAMPODULCE SRL*, espero que se encuentre bien, se le informa que se esta iniciando el trabajo de *PULVERIZACION* en la siguiente ubicacion: https://www.google.com/maps?q=-17.235721861668225,-62.97494216571519. Este mensaje a sido generado de manera automatica por el equipo de campo. Para cualquier consulta puede comunicarse con ING. ALEJANDRO SANCHEZ al 78448317
397;07-03-2025;01:01:32;41594;AGROP. CAMPODULCE SRL;68908131;Señor(a) *AGROP. CAMPODULCE SRL*, espero que se encue

In [ ]:
df_plan = pd.read_excel(path_xlsx_plan)

In [ ]:
df_plan = df_plan[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [ ]:
df_plan.head(3)

In [ ]:
# get data de nuevas notificacion de liberacion
query = 'select * from dataset_354655 where procesado=false'
notif = ejecutar_query_sql(33457, query, 'get')
notif = notif['data']

In [ ]:
notif

In [ ]:
# extrae el primer elemento
reg_amigo = notif[0]

In [ ]:
reg_amigo

In [ ]:
# extrae cod_ca y nom_ca del registro
cod_ca = int(reg_amigo['canhero'].split(' / ')[0])
nom_ca = reg_amigo['canhero'].split(' / ')[1]

In [ ]:
# filtra en la lista de planificacion el cod_ca para obtener los numeros cel.
reg = df_plan[df_plan['cod_ca']==cod_ca].iloc[0]

In [ ]:
# id del registro en amigocloud
idd = reg_amigo['id']

In [ ]:
idd

In [ ]:
# extrae fecha y hora de registro menos 4 horas por desfase temprotal
fecha_hora_obj = datetime.fromisoformat(reg_amigo['fecha_registro'])
fecha_hora_menos_4h = fecha_hora_obj - timedelta(hours=4)

In [ ]:
# formatea la fecha de registro
fecha_str = fecha_hora_menos_4h.strftime('%d-%m-%Y')

In [ ]:
# formatea la hora de registro
hora_str = fecha_hora_menos_4h.strftime('%H:%M:%S')

In [ ]:
# extrae los numeros telefonicos
telefonos = [reg['telf01'], reg['telf02'], reg['telf03']]
# valida los numeros telefonicos
telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]

In [ ]:
# extra el tipo de labor (liberacion o pulverizacion)
labor = reg_amigo['tipo_labor']

In [ ]:
# extrae el estado de la labor (inicio o fin de labor)
estado = reg_amigo['tipo_mensaje']

In [ ]:
# extrae nombre del piloto
tecnico = reg_amigo['piloto']

In [ ]:
# extrae y convierte la ubicacion
ubi = convertir_wkb(reg_amigo['ubicacion'])
longitud = ubi.x  # Longitud (coordenada X)
latitud = ubi.y   # Latitud (coordenada Y)

In [ ]:
crear_msj(idd, fecha_str, hora_str, cod_ca, nom_ca, telefonos_validos, labor, estado, tecnico, longitud, latitud)

In [ ]:
telefonos_validos